### Train a Regression Model Using Neural Network
In this activity, we will learn:
* use tensorflow keras to train and use a neural network model to estimate the price of a house 



### Create the Model  

First, we need to import all the libraries that we need and also mount our google drive. We’ll use several of the same scikit-learn helper functions, but we’ll also pull in a bunch of Keras classes. We’ll use Keras to define the neural network.

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib
from sklearn.metrics import mean_absolute_error
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# mount your google drive
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# change working director
# %cd "/content/drive/MyDrive/Colab Notebooks/goML"

Load our dataset and split it into X and y values using pandas exactly the same way we did in the last project. 

In [4]:
# Load our data set
df = pd.read_csv("data/house_data.csv")

In [5]:
# Create the X and y arrays
X = df[["sq_feet", "num_bedrooms", "num_bathrooms"]]
y = df[["sale_price"]]

In [6]:
X.head()

sq_feet  num_bedrooms  num_bathrooms
0      785             2              2
1     1477             2              2
2      712             1              1
3     3233             3              3
4     1581             2              1

But here’s where things start getting more complicated with neural networks.  All of our input and output data needs to be scaled to values between 0 to 1 for the neural network to train correctly. To do that, we will create two MinMaxScaler classes and tell them to scale values between 0 and 1: 

In [7]:
# Data needs to be scaled to  0 to 1 for the neural network to train correctly.
X_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(0, 1))

The reason we need to create a separate scaler for the input values and the output values is that we’ll want to be able to use them separately when we make real predictions with the model. 

In [8]:
# Scale both the training inputs and outputs
X[X.columns] = X_scaler.fit_transform(X[X.columns])
y[y.columns] = y_scaler.fit_transform(y[y.columns])

In [9]:
X.head()

sq_feet  num_bedrooms  num_bathrooms
0  0.081452      0.333333            0.5
1  0.279223      0.333333            0.5
2  0.060589      0.000000            0.0
3  0.781080      0.666667            1.0
4  0.308945      0.333333            0.0

We also need to convert the X and y arrays from pandas dataframes into numpy arrays. Newer versions of pandas provide a to_numpy() function on dataframes to make this easy.

This conversion doesn’t change the values of the data at all. The only reason we need to convert the datatype is because Keras runs on top of TensorFlow and TensorFlow 2.0 doesn’t support pandas dataframes directly. It’s a work-around for a capability issue. 

In [10]:
# Convert the inputs and outputs to numpy format so TensorFlow doesn't complain
X = X.to_numpy()
y = y.to_numpy()

Next, we’ll split the data into a training set and a test set. It’s important to do this after we scale the data to make sure that the training and test sets are scaled exactly the same way

In [11]:
# Split the data set in a training set (75%) and a test set (25%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

Now we are ready to define our neural network.  Keras has different APIs for defining neural networks that we can choose between. The most commonly used is the **Sequential API**. The Sequential API is the version in which we define it one layer at a time, in sequence. It’s the easiest way to do it, so that’s what we’ll use too. So, first we create a new Sequential() model. 

In [12]:
# Create a Neural Network model
model = Sequential()

This is the model object to which we’ll add our layers. Now we can define the layers of our neural network calling the **model.add()** function and passing in the kind of layer we want to add. 

So far, we’ve only learned about the simplest kind of neural network layer where every node in the layer is connected to every node in the following layer. These are called **dense** layers because the nodes are densely connected. 

To create a dense layer in Keras, we can create new **Dense()** objects and add them to the model.  Here’s how we’ll define the first layer: 


In [13]:
model.add(Dense(50, input_dim=3, activation='relu'))

This single line actually will create two layers—a three-node input layer and a 50-node hidden layer. This is because the first layer in Keras has a special input_shape parameter. This tells Keras that we want three input nodes in the neural network. Instead of declaring the input layer on its own line, this is how Keras specifies the size of the input layer.  The 50 parameter tells Keras to create the dense layer with 50 nodes. 

Earlier, we talked about hidden layers with four nodes. But in real life, we’ll often use hundreds of nodes in a single layer. 

Finally, each layer needs to have an **activation** function. This controls how much signal passes from each node in this layer to nodes in the next layer. Here are the two activation functions that we’ll use: 
* **relu** is a rectified linear unit. This activation function is non-linear but it has some nice mathematical properties that make it train more quickly than other activation functions. This is what you’ll see used most often these days. 
* **linear** is the same thing as having no activation function for the layer and letting the value for each node pass to the next layer unchanged. 


Next, let’s add two more hidden layers, one with 100 nodes and one with 50 nodes: 

In [14]:
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))

As usual, these hidden layers use a relu activation function. 

Now let’s add the output layer


In [15]:
model.add(Dense(1, activation='linear'))

Since we are only predicting one final value (the value of a house), we only need one output node. And since we are predicting a real-world value, we’ll ask it to use a linear activation function, which is the same thing as doing nothing. This is because we want the final prediction to be a linear number so we don’t need to do anything special to it. 

We are using the Keras API to define our neural network, but it uses TensorFlow behind the scenes to do all the math. Now that we’ve declared all the layers, we need to tell Keras to construct the neural network inside of TensorFlow for us. To do that, we’ll call the **compile()** function: 

In [16]:
model.compile(
    loss='mean_squared_error',
    optimizer='SGD'
)

We can also print a visual summary to see how the overall network looks like

In [17]:
model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                200       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 10,401
Trainable params: 10,401
Non-trainable params: 0
_________________________________________________________________


### Train the model

Now we’re ready to train the model. Keras models its syntax on scikit-learn, so its training function is also called fit(): 

In [18]:
# Train the model
model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=8,
    shuffle=True,
    verbose=2
)

Epoch 1/50
938/938 - 1s - loss: 0.0127
Epoch 2/50
938/938 - 1s - loss: 4.8493e-04
Epoch 3/50
938/938 - 1s - loss: 4.1352e-04
Epoch 4/50
938/938 - 1s - loss: 3.8889e-04
Epoch 5/50
938/938 - 1s - loss: 3.7501e-04
Epoch 6/50
938/938 - 1s - loss: 3.6553e-04
Epoch 7/50
938/938 - 1s - loss: 3.5871e-04
Epoch 8/50
938/938 - 1s - loss: 3.5262e-04
Epoch 9/50
938/938 - 1s - loss: 3.4688e-04
Epoch 10/50
938/938 - 1s - loss: 3.4251e-04
Epoch 11/50
938/938 - 1s - loss: 3.3962e-04
Epoch 12/50
938/938 - 1s - loss: 3.3587e-04
Epoch 13/50
938/938 - 1s - loss: 3.3522e-04
Epoch 14/50
938/938 - 1s - loss: 3.3195e-04
Epoch 15/50
938/938 - 1s - loss: 3.3151e-04
Epoch 16/50
938/938 - 1s - loss: 3.3018e-04
Epoch 17/50
938/938 - 1s - loss: 3.2971e-04
Epoch 18/50
938/938 - 1s - loss: 3.2773e-04
Epoch 19/50
938/938 - 1s - loss: 3.2768e-04
Epoch 20/50
938/938 - 1s - loss: 3.2582e-04
Epoch 21/50
938/938 - 1s - loss: 3.2576e-04
Epoch 22/50
938/938 - 1s - loss: 3.2462e-04
Epoch 23/50
938/938 - 1s - loss: 3.2445e-04
E

First, we pass in the training data and the matching answers for each training example. Then we have several parameters that we can control: 
*	**epochs** is how many times we will loop through the entire training dataset before ending the gradient descent training process. 
*	**batch_size** controls how many training examples are considered at once during each gradient descent update pass. 
*	**shuffle**=True tells Keras to randomize the order of the input data it sees. This is super important. You always want to randomize the order of your training data unless you know for sure that you have it stored in random order already. 
*	**verbose** just controls how much Keras prints on the screen during the training process. Setting it to 2 makes it print less output. 


Then we need to save both scalers to a file since we will need them again later to make predictions. 

In [19]:
# Save the scalers to files so we can use it to pre-process new data later
joblib.dump(X_scaler, "models/X_scaler.pkl")
joblib.dump(y_scaler, "models/y_scaler.pkl")

['models/y_scaler.pkl']

Then we’ll save the trained neural network itself to a file so that we can use it to make predictions later

In [20]:
# Save the trained model to a file so we can use it to make predictions later
model.save("models/house_value_nn_model.h5")

And just like when we used the linear regression model, we will print out training accuracy results. The only difference here is we’ll use the y_scaler to unscale the accuracy numbers so that the results are in dollar amounts instead of a number between 0 and 1: 

In [21]:
# Report how well the model is performing
print("Model training results:")

# Report mean absolute error on the training set in a value scaled back to dollars so it's easier to
#  understand.
predictions_train = model.predict(X_train, verbose=0)

mse_train = mean_absolute_error(
    y_scaler.inverse_transform(predictions_train),
    y_scaler.inverse_transform(y_train)
)
print(f" - Training Set Error: {mse_train}")


Model training results:
 - Training Set Error: 9163.691315625


That covers the training data set results. We will do the same for the test data set. 

In [22]:
# Report mean absolute error on the test set in a value scaled back to dollars so it's easier to 
# understand.
predictions_test = model.predict(X_test, verbose=0)

mse_test = mean_absolute_error(
    y_scaler.inverse_transform(predictions_test),
    y_scaler.inverse_transform(y_test)
)
print(f" - Test Set Error: {mse_test}")


 - Test Set Error: 9365.347678125


In the end, we got nearly the same results as we got using the linear regression model, but the training process took a lot longer. This is because neural networks are much more complex mathematically than linear regression models so they take more computational power to train. 
This data set is really simple, so we didn’t really need the extra modelling power that the neural network provided to solve it. But in lots of cases with real-world data, a linear regression model won’t work at all because the data is too complex. In that case, the extra processing time required for a neural network would be worth it. 

Also, notice that the training set and test set errors are different. Models nearly always do worse on the test data than the training data. That’s because the model can cheat and memorize answers in the training dataset, but it can’t do that with the test data since it never sees those answers. 
That means that Test Set Error is a much better predictor of real-world performance than Training Set Error. But the distance between the two numbers also gives us tons of clues of how well your model is working: 

* If the Test Set Error is reasonably low and only a little higher than your Training Set Error, then everything is working correctly! 
* If your Training Set Error and Test Set Error are both really high, the model is failing to learn at all. This is called **underfitting**. 
* If your Training Set Error is low but your Test Set Error is high, the model is memorizing the training data and not really learning anything. This is called **overfitting**. 

**Overfitting** and **underfitting** are the bane of machine learning. Training a model is a balancing act where you are trying to find the best spot between those two conditions. 
There are several strategies you can use to fix underfitting and overfitting. 

Strategies for fixing underfitting: 
*	Make sure the output value that you want to predict can reasonably be determined by the training inputs. Machine learning isn’t magic! 
*	Try running the training process longer. 
*	Increase model complexity by adding layers or adding nodes to layers. 
*	Try a different machine learning algorithm that is better suited to the problem. 

Strategies for fixing overfitting: 
*	Reduce the model complexity by removing layers or reducing the number of nodes in layers. 
*	Use regularization techniques like dropout. 
*	Increase the amount of training data. 

It’s important to be systematic when evaluating a model. Following these guidelines will save you a lot of time and money. For example, there’s absolutely no reason to collect more training data if your model is underfitting. In that case, no amount of extra data will help! Instead, make sure your model can overfit the training data you already have before adding more data.


### Using the model

Now that we’ve trained the neural network, let’s use it to make new predictions from a new Python program. 
First, we need to import some libraries so we can load the model file and the data scalers that we created earlier. 


In [23]:
from tensorflow.keras.models import load_model
import joblib

Load the model that we just trained. 

In [24]:
# Load our trained model
model = load_model('models/house_value_nn_model.h5')

Load the data scalers. 

In [25]:
# Load the data scalers so that we can transform new data and prediction the same way as training data.
X_scaler = joblib.load('models/X_scaler.pkl')
y_scaler = joblib.load('models/y_scaler.pkl')

Now let’s specify the house that we want to value. 

In [26]:
#Define the house that we want to value (with the values in the same order as in the training data).
house_1 = [
    2000,  # Size in Square Feet
    3,  # Number of Bedrooms
    2,  # Number of Bathrooms
]

Keras assumes we want to make multiple price predictions at once, so we need to put our single house inside an array. 

In [27]:
# Keras assumes we want to predict the values for multiple of houses at once, so it expects an array.
# We only want to value a single house, so it will be the only item in our array.
homes = [
    house_1
]

But we need to remember that the house we just defined needs to have those values scaled to values between 0 and 1 so the model will understand the values. We can do that using the X_scaler we created during the training process. 

In [28]:
# Scale the new data like the training data
scaled_home_data = X_scaler.transform(homes)

C:\Users\seow_khee_wei\Anaconda3\envs\micron\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Prediction in Keras has nearly the same syntax as in scikit-learn. Just call model.predict() and pass in the data for the house. 

In [29]:
# Make a prediction for each house in the homes array (but we only have one)
home_values = model.predict(scaled_home_data)

The prediction we get back from the neural network will be a value between 0 and 1. To turn it back into a dollar amount, we need to undo the scaling. The y_scaler we created during training can do that with its inverse_transform() function. 

In [30]:
# The prediction from the neural network will be scaled 0 to 1 just like the training data
# We need to unscale it using the same factor as we used to scale the training data
unscaled_home_values = y_scaler.inverse_transform(home_values)

Now we can just print out the results. 

In [31]:
# Since we are only predicting the price of one house, grab the first prediction returned
predicted_value = unscaled_home_values[0][0]

# Print the results
print("House details:")
print(f"- {house_1[0]} sq feet")
print(f"- {house_1[1]} bedrooms")
print(f"- {house_1[2]} bathrooms")
print(f"Estimated value: ${predicted_value:,.2f}")

House details:
- 2000 sq feet
- 3 bedrooms
- 2 bathrooms
Estimated value: $382,902.22


Try playing around with the characteristics of the house and seeing what values it will predict. You can also compare the neural network’s predictions against the linear regression model we made earlier to see where they agree and where they disagree. 

### Exercise
Add another hidden layer of 100 nodes and see if your MSE improves.

### Activity wrap up

We learnt how to
* use tensorflow keras to train and use a neural network model to estimate the price of a house

